In [1]:
'''
Notebook for the main text's FIGS?: Simulating cell cycle noise with variable cell volumes
'''
# By Kirill Sechkar

# PACKAGE IMPORTS 
import numpy as np
import jax
import jax.numpy as jnp
import functools
from diffrax import diffeqsolve, Dopri5, ODETerm, SaveAt, PIDController, SteadyStateEvent
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import time

# set up jax
from jax.lib import xla_bridge
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
print(xla_bridge.get_backend().platform)

# set up bokeh
bkio.reset_output()
bkio.output_notebook() 

# OWN CODE IMPORTS
import synthetic_circuits as circuits
from cell_model import *

/tmp/ipykernel_14438/75946051.py:21: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  print(xla_bridge.get_backend().platform)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


cpu


Loading BokehJS ...

In [2]:
# INITIALISE CELL MODEL, LOAD THE CIRCUIT

# initialise cell model
cellmodel_auxil = CellModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = cellmodel_auxil.default_params()  # get default parameter values
init_conds = cellmodel_auxil.default_init_conds(par)  # get default initial conditions

# load synthetic gene circuit
ode_with_circuit, circuit_F_calc, circuit_eff_m_het_div_k_het,\
    par, init_conds, circuit_genes, circuit_miscs, circuit_name2pos, circuit_styles, circuit_v = cellmodel_auxil.add_circuit(
    circuits.punisher_cnc_b_initialise,
    circuits.punisher_cnc_b_ode,
    circuits.punisher_cnc_b_F_calc,
    circuits.punisher_cnc_b_eff_m_het_div_k_het,
    par, init_conds,
    circuits.punisher_cnc_b_v_varvol)  # load the circuit

In [3]:
# PARAMETERISE THE CIRCUIT

# BURDENSOME SYNTHETIC GENE
par['c_b'] = 1
par['a_b'] = 1e5

# PUNISHER
# switch gene conc
par['a_switch'] = 400.0  # promoter strength (unitless)
par['d_switch'] = 0.01836
# integrase - expressed from the switch gene's operon, not its own gene => c_int, a_int irrelevant
par['k+_int'] = par['k+_switch'] / 80.0  # RBS weaker than for the switch gene
par['d_int'] = 0.0  # 0.01836 # rate of integrase degradation per protease molecule (1/nM/h)
# CAT (antibiotic resistance) gene
par['a_cat'] = 500.0  # promoter strength (unitless)
par['n_cat'] = 300.0
# synthetic protease gene
par['a_prot'] = 25.0  # promoter strength (unitless)
init_conds['p_prot'] = 1500.0  # if zero at start, the punisher's triggered prematurely

# punisher's transcription regulation function
par['K_switch'] = 300.0  # Half-saturation constant for the self-activating switch gene promoter (nM)
par['eta_switch'] = 2  # Hill coefficient for the self-activating switch gene promoter (unitless)
par['baseline_switch'] = 0.025  # Baseline value of the switch gene's transcription activation function
par['p_switch_ac_frac'] = 0.85  # active fraction of protein (i.e. share of molecules bound by the inducer)

# plasmid copy number control
init_conds['cat_pb'] = 10.0  # INITIAL CONDITION (not a parameter): all plasmids have working CAT gene copies
par['k_tr'] = 130.2  # plasmid replication rate (1/h)
par['a_inh'] = 948  # inhibitor synthesis rate per plasmid copy (1/h)
par['b_inh'] = 74.976  # inhibitor degradation rate (1/h)
par['n_inh'] = 10  # number of steps of replication initiation at which inhibition can happen
par['K_inh'] = 214.05  # replication inhibition constant (nM)

# critical cell volume triggering division
par['V_crit'] = 2.0*np.log(2)   # 2ln(2) so as to have an average volume of 1 um^3 assuming constant growth rate

# par['k_sxf']=0.0
# par['d_switch']=0.0
# # par['a_b']=0.0
# par['a_prot']=0.0
# par['a_cat']=0.0
# par['a_switch']=0.0

# culture medium
nutr_qual=0.5
par['s'] = nutr_qual  # nutrient quality (unitless)
init_conds['s'] = nutr_qual  # nutrient quality (unitless)

In [4]:
# DETERMINISTIC SIMULATION TO FIND THE STARTING STEADY STATE - SET SIMULATION PARAMETERS
# set simulation parameters
tf = (0, 50)  # simulation time frame
savetimestep = 0.1  # save time step
dt_max = 0.1  # maximum integration step
rtol = 1e-6  # relative tolerance for the ODE solver
atol = 1e-6  # absolute tolerance for the ODE solver

In [5]:
# DETERMINISTIC SIMULATION TO FIND THE STARTING STEADY STATE - RUN

sol = ode_sim(par,  # dictionary with model parameters
              ode_with_circuit,  # ODE function for the cell with synthetic circuit
              cellmodel_auxil.x0_from_init_conds(init_conds, circuit_genes, circuit_miscs),
              # initial condition VECTOR
              len(circuit_genes), len(circuit_miscs), circuit_name2pos,
              # dictionaries with circuit gene and miscellaneous specie names, species name to vector position decoder
              cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
              # synthetic gene parameters for calculating k values
              tf, jnp.arange(tf[0], tf[1], savetimestep),  # time axis for saving the system's state
              rtol,
              atol)  # simulation parameters: when to save the system's state, relative and absolute tolerances)   # simulation parameters: time frame, save time step, relative and absolute tolerances
ts = np.array(sol.ts)
xs = np.array(sol.ys)
# det_steady_x = jnp.concatenate((sol.ys[-1, 0:8], jnp.round(sol.ys[-1, 8:])))
det_steady_x = sol.ys[-1, :]

In [6]:
# HYBRID SIMULATION WITH VARIABLE CELL VOLUME
# tau-leap hybrid simulation parameters
tf_hybrid = (tf[-1], tf[-1] + 0.3)  # simulation time frame
tau = 1e-7  # simulation time step
tau_odestep = 1e-7  # number of ODE integration steps in a single tau-leap step (smaller than tau)
tau_savetimestep = 1e-2  # save time step a multiple of tau

# simulate
timer = time.time()
mRNA_count_scales, S, x0_tauleap, circuit_synpos2genename, keys0 = tauleap_sim_prep_varvol(par, len(circuit_genes),
                                                                                    len(circuit_miscs),
                                                                                    circuit_name2pos, det_steady_x,
                                                                                    key_seeds=[0]
                                                                                    )
x0_tauleap[6]=1.0 # start at the default volume of 1 um^3
ts_jnp, xs_jnp, final_keys = tauleap_sim_varvol(par,  # dictionary with model parameters
                                         circuit_v,  # circuit reaction propensity calculator
                                         circuit_eff_m_het_div_k_het,
                                         x0_tauleap,
                                         # initial condition VECTOR (processed to make sure random variables are appropriate integers)
                                         len(circuit_genes), len(circuit_miscs), circuit_name2pos,
                                         cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),
                                         # synthetic gene parameters for calculating k values
                                         tf_hybrid, tau, tau_odestep, tau_savetimestep,
                                         # simulation parameters: time frame, tau leap step size, number of ode integration steps in a single tau leap step
                                         mRNA_count_scales, S, circuit_synpos2genename,
                                         # mRNA count scaling factor, stoichiometry matrix, synthetic gene number in list of synth. genes to name decoder
                                         keys0, avg_dynamics=True)  # starting random number genereation key

# concatenate the results with the deterministic simulation
ts = np.concatenate((ts, np.array(ts_jnp)))
xs_first = np.concatenate(
    (xs, np.array(xs_jnp[0])))  # getting the results from the first random number generator key in vmap
xss = np.concatenate((xs * np.ones((keys0.shape[0], 1, 1)), np.array(xs_jnp)),
                     axis=1)  # getting the results from all vmapped trajectories

In [7]:
# PLOT THE CELL MODEL TRAJECTORIES

# PLOT - HOST CELL MODEL
# bkplot.output_file(filename="cellmodel_sim.html",
#                    title="Cell Model Simulation")  # set up bokeh output file
mass_fig = cellmodel_auxil.plot_protein_masses(ts, xs_first, par, circuit_genes)  # plot simulation results
nat_mrna_fig, nat_prot_fig, nat_trna_fig, h_fig = cellmodel_auxil.plot_native_concentrations(ts, xs_first, par,
                                                                                             circuit_genes,
                                                                                             tspan=tf_hybrid,
                                                                                             varvol=True)  # plot simulation results
l_figure, e_figure, Fr_figure, ppGpp_figure, nu_figure, D_figure = cellmodel_auxil.plot_phys_variables(ts,
                                                                                                       xs_first,
                                                                                                       par,
                                                                                                       circuit_genes,
                                                                                                       circuit_miscs,
                                                                                                       circuit_name2pos,
                                                                                                       circuit_eff_m_het_div_k_het,
                                                                                                       tspan=tf_hybrid,
                                                                                                       varvol=True)  # plot simulation results
vol_figure = cellmodel_auxil.plot_volume(ts, xs_first, par, circuit_genes, tspan=tf_hybrid)  # plot simulation results
bkplot.show(bklayouts.grid([[nat_mrna_fig, nat_prot_fig, vol_figure],
                            [nat_trna_fig, h_fig, l_figure],
                            [e_figure, Fr_figure, D_figure]]))

In [8]:
# PLOT THE CIRCUIT TRAJECTORIES

# bkplot.output_file(filename="circuit_sim.html",
#                    title="Synthetic Gene Circuit Simulation")  # set up bokeh output file
het_mrna_fig, het_prot_fig, misc_fig = cellmodel_auxil.plot_circuit_concentrations(ts, xs_first, par,
                                                                                   circuit_genes,
                                                                                   circuit_miscs,
                                                                                   circuit_name2pos,
                                                                                   circuit_styles, tspan=tf_hybrid,
                                                                                   varvol=True)  # plot simulation results
F_fig = cellmodel_auxil.plot_circuit_regulation(ts, xs_first, circuit_F_calc, par, circuit_genes, circuit_miscs,
                                                circuit_name2pos, circuit_styles, tspan=tf_hybrid,
                                                varvol=True)  # plot simulation results
bkplot.show(bklayouts.grid([[het_mrna_fig, het_prot_fig, misc_fig],
                            [F_fig, None, None]]))

In [9]:
# PLOT TOTAL PER-CELL ABUNDANCES

nat_mrna_fig, nat_prot_fig, nat_trna_fig, h_fig = cellmodel_auxil.plot_native_concentrations(ts, xs_first, par,
                                                                                             circuit_genes,
                                                                                             tspan=tf_hybrid,
                                                                                             varvol=False)  # plot simulation results
bkplot.show(bklayouts.grid([[nat_mrna_fig, nat_prot_fig, nat_trna_fig]]))

In [10]:
het_mrna_fig, het_prot_fig, misc_fig = cellmodel_auxil.plot_circuit_concentrations(ts, xs_first, par,
                                                                                   circuit_genes,
                                                                                   circuit_miscs,
                                                                                   circuit_name2pos,
                                                                                   circuit_styles, tspan=tf_hybrid)  # plot simulation results
bkplot.show(bklayouts.grid([[het_mrna_fig, het_prot_fig, misc_fig]]))

In [11]:
# PLOT THE TRANSLATION RATE

x_prediv=xs_first[523,:]
x_div,_=tauleap_division_varvol(x_prediv,par,final_keys[0],True)

tu_counts=xs_first[:,5]
tu_concs=tu_counts/xs_first[:,6]
Vs=xs_first[:,6]

print(Vs[523],Vs[524])
print(tu_counts[524]/tu_counts[523])
print(tu_concs[524]/tu_concs[523])

1.3629959249059174 1.3814721830137082
1.0135556519939075
1.0000000291857833


In [12]:
tc_counts=xs_first[:,4]
tc_concs=tc_counts/xs_first[:,6]

print(tc_counts[524]/tc_counts[523])
print(tc_concs[524]/tc_concs[523])

pa_counts=xs_first[:,2]
pa_concs=pa_counts/xs_first[:,6]

print(pa_counts[524]/pa_counts[523])
print(pa_concs[524]/pa_concs[523])

pb_counts=xs_first[:,circuit_name2pos['p_b']]
pb_concs=pb_counts/xs_first[:,6]

print(pb_counts[524]/pb_counts[523])
print(pb_concs[524]/pb_concs[523])

1.0135548190295385
0.9999992073617506
1.013555489316245
0.9999998686838248
1.013555057716334
0.9999994428562713


In [13]:
args = (
        par,  # model parameters
        circuit_name2pos,  # gene name - position in circuit vector decoder
        len(circuit_genes), len(circuit_miscs),  # number of genes and miscellaneous species in the circuit
        cellmodel_auxil.synth_gene_params_for_jax(par, circuit_genes),  # relevant synthetic gene parameters in jax.array form
        mRNA_count_scales, S, circuit_synpos2genename  # parameters for stochastic simulation
    )
stoch_update_prediv=np.array(tauleap_update_stochastically_varvol(0, x_prediv, tau, args, circuit_v, final_keys[0], avg_dynamics=True))
stoch_update_div=np.array(tauleap_update_stochastically_varvol(0, x_div, tau, args, circuit_v, final_keys[0], avg_dynamics=True))

print(stoch_update_prediv)
print(stoch_update_div)


[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -1.18753574e+01  1.18753574e+01  0.00000000e+00  0.00000000e+00
  3.36454818e-03  4.76803846e-06  0.00000000e+00  1.68453600e-04
  8.06047449e-06  3.75623038e-02  6.50183406e-06  6.40264782e-07
  1.87566796e-03  9.46914056e-05  1.83082936e-06  8.78073858e-12
  8.44968181e-12  2.27389270e-05]
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -5.93767870e+00  5.93767870e+00  0.00000000e+00  0.00000000e+00
  1.68227409e-03  2.38401923e-06  0.00000000e+00  8.42268002e-05
  4.03023724e-06  1.87811519e-02  3.25091703e-06  3.20132391e-07
  9.37833981e-04  4.73457028e-05  9.15414679e-07  4.39036929e-12
  4.22484090e-12  1.13694635e-05]
